This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sample = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key={API_KEY}')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample.text

'{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2019-10-30","frequency":"daily","data":[["2019-10-30",96.45,98.85,96.0,98.05,null,101101.0,9910827.0,null,null,null]],"collapse":null,"order":null}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
'''
1. Collect data from the Franfurt Stock Exchange, for the ticker 
AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
'''

# Get the date using an API call.
# I'll split up the call into multiple sections for readability.
start_date = '2017-01-01'
end_date = '2017-12-31'
dataset_url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
api_call = f'{dataset_url}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'

# Get the JSON data using the requests package
response = requests.get(api_call)
response

<Response [200]>

In [6]:
'''
2. Convert the returned JSON object into a Python dictionary.
'''

# Luckily, the requests library has a function json() that does just that.
data = response.json()
type(data)

dict

In [7]:
# For future reference, I'll check out the keys of the returned dictionary.
data.keys()

dict_keys(['dataset_data'])

In [8]:
# Seems like we need to go one level deeper.
data['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [9]:
# Finally I'll see what columns are in the dataset.
data['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [10]:
'''
3. Calculate what the highest and lowest opening prices were for the stock in this period.
'''
# First I'll need the price data.
price_data = data['dataset_data']['data']

# I'll also need the index of the opening prices column.
open_index = data['dataset_data']['column_names'].index('Open')

# A simple generator expression can be used to find the max value.
max_opening_price = max(date[open_index] for date in price_data if date[open_index])
max_opening_price

53.11

In [11]:
# The minimum value.

In [12]:
min_opening_price = min(date[open_index] for date in price_data if date[open_index])
min_opening_price

34.0

In [13]:
'''
4. What was the largest change in any one day (based on High and Low price)?
'''
# For this problem I'll need the indexes of the High and Low price columns.
high = data['dataset_data']['column_names'].index('High')
low = data['dataset_data']['column_names'].index('Low')

# A generator expression can then be used to find the largest one day price change.
max_single_day_price_change = max(date[high] - date[low] for date in price_data)
max_single_day_price_change

2.8100000000000023

In [14]:
'''
5. What was the largest change between any two days (based on Closing Price)?
'''
# Python dictionaries, unlike most language's dictionaries, are ordered by default as of Python 3.5.
# However, assuming that the version of Python is older,
# I will need another way to calculate inter-row operations.
# There is a parameter found in the quandl API documentation that performs row to row calculations beforehand.

# I'll update the api call to add the 'transform' parameter with a value of 'diff' to find row-on-row change.
diff_api_call = f'{dataset_url}?transform=diff&start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
diff_response = requests.get(diff_api_call)
diff_data = diff_response.json()

# Now the price_data will contain the difference in value from the previous day. 
diff_price_data = diff_data['dataset_data']['data']
close_index = diff_data['dataset_data']['column_names'].index('Close')

# Another generator expression finds the maximum value.
max_day_to_day_change = max(date[close_index] for date in diff_price_data)
max_day_to_day_change

1.72

In [15]:
'''
6. What was the average daily trading volume during this year?
'''

# I'll return to the original dataset for this problem. I will need the trading volume column index.
price_data = data['dataset_data']['data']
traded_volume = data['dataset_data']['column_names'].index('Traded Volume')

# Divide the sum of values by the number of values to find the average.
average_traded_volume = sum(date[traded_volume] for date in price_data) / len(price_data)
average_traded_volume

89124.33725490196

In [16]:
'''
7. (Optional) What was the median trading volume during this year.
(Note: you may need to implement your own function for calculating the median.)
'''

# To start with, I'll write up a general median function that will accept any iterable that can be sorted.
# It does not accept generator expressions as they do not have a defined len() value
def median(iterable):
    # Guard against empty lists.
    if not iterable:
        return None
    # Sort the data.
    s = sorted(iterable)
    # Find the middle value. The below expression works on both even and odd lengths.
    middle = (s[(len(s) - 1) // 2] + s[len(s) // 2]) / 2
    return middle

# Let's run a few tests.
assert median([5, 3, 2]) == 3
assert median([4]) == 4
assert median([8, 7, 8, 7]) == 7.5
assert median([]) == None

In [17]:
# Everything looks good, so I'll use it on the trading volume data for the solution.
median_traded_volume = median([date[traded_volume] for date in price_data])
median_traded_volume

76286.0